# Overview
In this notebook we explore the basics of spark and get ready to put hands on keyboard.

While Spark provides a helpful [gloassary](https://spark.apache.org/docs/3.2.0/cluster-overview.html#glossary) and some nice documentation, in my humble opinion it is not comprehensive. Below I try to uncover and tie together all the important concepts of spark.

We will start by understanding what spark can do and the features it offers. We then talk about the object model to understand how data is represented and how it is manipulated. We then talk about the software architecture to really understand how our python code is translated into code that the cluster understands. We when briefly look at the cluster architecture to understand how the work is carried out.

# 1. Spark Features

Spark provides several key features:

- Spark SQL - Many data scientists, analysts, and general business intelligence users rely on interactive SQL queries for exploring data. Spark SQL is a Spark module for structured data processing. It provides a programming abstraction called DataFrames and can also act as distributed SQL query engine. It enables unmodified Hadoop Hive queries to run up to 100x faster on existing deployments and data. It also provides powerful integration with the rest of the Spark ecosystem (e.g., integrating SQL query processing with machine learning).

- Spark Streaming - Many applications need the ability to process and analyze not only batch data, but also streams of new data in real-time. Running on top of Spark, Spark Streaming enables powerful interactive and analytical applications across both streaming and historical data, while inheriting Spark’s ease of use and fault tolerance characteristics. It readily integrates with a wide variety of popular data sources, including HDFS, Flume, Kafka, and Twitter.

- MLlib - Machine learning has quickly emerged as a critical piece in mining Big Data for actionable insights. Built on top of Spark, MLlib is a scalable machine learning library that delivers both high-quality algorithms (e.g., multiple iterations to increase accuracy) and blazing speed (up to 100x faster than MapReduce). The library is usable in Java, Scala, and Python as part of Spark applications, so that you can include it in complete workflows.

- GraphX - GraphX is a graph computation engine built on top of Spark that enables users to interactively build, transform and reason about graph structured data at scale. It comes complete with a library of common algorithms.

We will see that these are tied to the APIs listed in the following section

# 2. Spark APIs

The Spark Project offers several classes of features. As such, the code base can be broken up into several coresponding APIs. 

The first few deal with the data structures. We will encounter these first.

- RDDs
- DataFrames
- Datasets

The next deal with the analyitcs and machine learning that can be performed on the data structures. We will likely encounter these second, once we have our data in some type of data structure.

- Spark SQL
- Spark Streaming
- MLlib
- GraphX

And lastely we have the the API which deals with the underlying system. This is something we will likely not encounter as a data scientist.

- Spark Core

   Spark Core is the underlying general execution engine for the Spark platform that all other functionality is built on top of. It provides in-memory computing capabilities to deliver speed, a generalized execution model to support a wide variety of applications, and Java, Scala, and Python APIs for ease of development.

# 3. Object Model

In this section we look at some of the objects spark Provides as part of it's APIs

In [ ]:
Action vs operation?
https://stackoverflow.com/questions/31508083/difference-between-dataframe-dataset-and-rdd-in-spark

## 3.1. Data Abstractions
There are several way to represent data within Apache spark:

The TL;DR; Is that we will be using dataframes as 

For more information, you can watch this great [video](https://www.youtube.com/watch?v=Ofk7G3GD9jk) on these structures which was recorded from the Spark Summit organized by databricks.

### 3.1.1. Resilient Distributed Datasets (RDD)

The (Resilient Distributed Dataset) RDD was the primary user-facing API in Spark since its inception. The RDD API has been in Spark since the 1.0 release. It provides the lowest level API to control the structure and transformations of data.

An RDD has the following properties:
- Distributed - the data is logically sharded into partitions and stored accross multiple machines while presented as a single element
- Resiliant - rdd can be recreated at any point in time during its lifetime (including stages alond execution). For example, if somehting goes wrong, we can revert.
- Immutable - original version remains intact from transformations etc
- Compile-Time Safe - saves time early in the process of submitting an application
- Supports structured and unstructured data
- Lazy - Dont materialize until an Action is performed (Action is a DAG of Transformations on an RDD)

These properties allow a few key features:

1. Because the data is partitioned across nodes in your cluster it that can be operated on in parallel 
2. Because the RDD is imutable and Resiliant we can create a Directed Asyclic Graph (DAG) to represent the linear flow of data as it goes through a transformative process.

There is a tradeoff with being at such a low level in the stack:
- User expresses how to do something not what to do
- Not optimized by Spark, user must optimize or suffer inefficiencies
- Slow for non JVM languages (like python or R)

### 3.1.2. Dataframes
Spark 1.3 introduced a new DataFrame API as part of the Project Tungsten initiative which seeks to improve the performance and scalability of Spark. The DataFrame is built on top of the RDD API. It offers a lot of benefits to users who do not want to concern themselves with the low level details of the RDD API. With the DataFrame you use a declarative syntax to say what you want to do and the Spark Framework will develop a query plan using an optimizer which determines how to do something.

The DataFrame is the main type of object we will interact with as data scientists using python.

In some documentation the DataFrame is referred to as the first of the Structured APIs offered by spark. this is because the DataFrame API introduces the concept of a schema to describe the data. In doing so, the DataFrame is able to detect syntax errors at compile time. This is a huge benefit as we can catch problems in the code before a long running batch starts executing. Additionally this feature allows Spark to pass data between nodes in a much more efficient way than using Java serialization.


### 3.1.3. Datasets
The Dataset API, released as an API preview in Spark 1.6. The Dataset, like the DataFrame is built on top of the RDD API. It offers compile time safety checks for syntax errors like the DataFrame and in addition it offers compile time checks for errors in analysis.

The Dataset aims to provide the best of both worlds; the familiar object-oriented programming style and compile-time type-safety of the RDD API but with the performance benefits of the Catalyst query optimizer. Datasets also use the same efficient off-heap storage mechanism as the DataFrame API.

In spark 2.0 the APIs were unified so that developers using DataFrames were actually using aliases to DataSets.


### 3.1.4. Cache
Apache spark provides a cluster-wide in-memory [cache](https://spark.apache.org/docs/latest/quick-start.html#caching). This is very useful when data is accessed repeatedly, such as when querying a small “hot” dataset or when running an iterative algorithm like PageRank.

Unlike Broadcast Variables, the cached variables are mutable.



### 3.2.5. Broadcast Variables
As the name suggests, Broadcast Variables are Variables or objects you define using the Spark API. The difference between broadcast variables and other variables, like the DataFrame, is that the Broadcast Variables
are read-only and are copied to and cached on all nodes in a cluster.

This is useful in some cases as it reduces the amount of data transfer that needs to take place for certain operations. Consider a case where we would like to do a lookup; we need join two data sets (one large, one small) and return a column from each as a final DataFrame. The Join operation will cause the data to be shuffled around so that the right pieces are on the right nodes so that the join can take place. Instead, we can Broadcast the small dataset accross the cluster. The Join operation can then takeplace without the overhead of moving the large data set (we only move the small one).

There is a limitation to the size of a broadcase variable (with Spark 3.0 I think it is ~2GB) which means they should not be used for large datasets.

## 3.2. Execution Abstractions

It's important to know some of the basic terminology related to how spark is executing work so that we understand the dashboard and monitoring etc.

As a data scientist, mostly we will be calling functions on a DataFrame in python etc. This function call will translate down the layers of the software stack until we hit the cluster layer. As we do so, we leave a declarative syntax which expresses what we would like to do and begin talking about how we would like to do things. With that being said, we know the RDD is the basic building block of the DATA related APIs so we wil start there.

The RDD provides two object-oritented means of manipulating the underyling data that the RDD encapsulates:
- **Transformations** - A lazy function that produces a new RDD. Transformation are executed when we call an Action. Examples inclue *map()* and *filter()* methods.
- **Actions** - A non-lazy function that produces a non-RDD result. For example *count()* returns an integer value immediately. The Transformations applied to an RDD create a DAG which must be executed before the Action's result can be determined. Thus an Action triggers the Execution declared by the Transformations.

As the RDD is an abstraction, the actions it is performing are ultimately passed on to the low level components of the Spark cluster that perform the low level operations.

- **Jobs** - Every Action schedules a Job to execute on accross the workers. As an action triggers the DAG of Transformations, a Job is a series of Stages.
- **Stage** - A Stage is a set of Tasks.
- **Tast** - A Task is the smallest unit of work. A Task coresponds to the command sent by the driver. Each command requires that it is performed on each partition of the data. Thus we will have a task for each partition.

# 4. Software Architecture

## 4.1. Application

While never explicitly defined, a spark application is an executable (code file or snippet) which leverages the Spark API to utlize a Spark cluster. In order to be executed, a Spark Application is submitted to a Spark cluster using one of the two execution modes described below. As mentioned in the [README](README.md) Spark provides APIs in Java, Scala, R, and Python which allow users to define their applciaitons. As we will see, these Applications leverage Spark Entry Points.

## 4.2. Java and The JVM
Spark runs on the JVM. To understand the spark architecture we need to understand this.

Spark is written in java and scala. Java runs natively in the JVM while scala can be comiled into Java bytecode and run inside the JVM.

The Java Virtual Machine (JVM) is an engine (sometimes called a virtual machine ... confusingly named) which provides the java runtime environment (JRE). In order to execute java code, it must be compiled into a jar file and executed inside the JVM. The Java Development Kit (JDK) allows the java code to access the JRE once inside the JVM.

## 4.3. Py4j
Py4j is a python library which allows python to interact with spark. It is the backbone of the pyspark API.

On the [homepage](https://www.py4j.org/index.html) Py4j is described as:

> A Bridge between Python and Java
>
> Py4J enables Python programs running in a Python interpreter to dynamically access Java objects in a Java Virtual Machine. Methods are called as if the Java objects resided in the Python interpreter and Java collections can be accessed through standard Python collection methods. Py4J also enables Java programs to call back Python objects.

Reading a bit deeper into the [documentation](https://www.py4j.org/getting_started.html) we see that the interop between python and java is provided by a **GatewayServer** instance. This gateway server allows Python programs to communicate with the JVM through a local network socket and send it instructions. It also has a callback functionality so that objects in the JVM can update objects in python in an event based manner. This Gateway is referred to as the Java Gateway in various points in the source code.

This is very important. Recall that the Driver runs in the JVM. If we look at the pyspark source code on [github](https://github.com/apache/spark/blob/e91ef1929201d4e493bb451fef0fb1b45800adae/python/pyspark/java_gateway.py#L214) we can see that a Driver is created in the JVM and a python wrapper provided by py4j and the Java Gateway allows us to manipulate the driver.

## 4.4. Spark Shell
The Spark Shell is an analog of the traditional operating system shell (like BASH, CMD, or PowerShell). The Spark shell provides an interactive command line interface (CLI) through which a user can interact with the Spark API and thus a Spark Cluster (once properly configured). The Spark Shell's CLI allows users to type and execute ad-hoc lines of Scala, Python, or R code. Each language specific shell has its own name, for example the python Spark Shell is sometimes called the **pyspark shell**. Like any Shell, the Spark Shell follows the REPL (read-evaluate-print loop) pattern.

An important point of note is that the Spark Shell acts as the Driver while a user incrementally defines their Application using the CLI (note: I am using the term Application very precisely. See Application defined alsewhere in this document.). We will see later that the Spark Shell is leveraged in a few ways.

More on the Spark Shell can be found in the official [documentation](https://spark.apache.org/docs/latest/quick-start.html).

## 4.5. Spark Conf
The SparkConf object configures the SparkContext and Driver. Spark provides a [list of configurations for the SparkContext](https://spark.apache.org/docs/latest/configuration.html).

## 4.6. Entry Points
As mentioned previously, users develop applications which leverage Apache Spark. As such, many articles and documentations talk about the entry point of a Spark application. This was a bit confusing; traditionally applications are said to have entry points or *main()* functions. These entrypoints are the gateway where the execution of an application begins and the user gets access to the runtime environment and/or API that the applciation was built to interact with. With Spark, the entrypoint gives the program access to the Spark environment.

In Spark 1.x, three entry points were introduced: SparkContext, SQLContext and HiveContext. Since Spark 2.x, a new entry point called SparkSession has been introduced that essentially combined all functionalities available in the three aforementioned contexts. Note that all contexts are still available even in newest Spark releases, mostly for backward compatibility purposes.

More information on these entrypoints can be found [here](https://towardsdatascience.com/sparksession-vs-sparkcontext-vs-sqlcontext-vs-hivecontext-741d50c9486a).

### 4.6.1. SparkSession
This object holds references to all the other Context objects.

### 4.6.2. SparkContext

The SparkContext is an integral part of Spark. Unfortunately There is no explicit definition anywhere in the official documentation and there are a lot of confusing, contradicting, muttled definitions floating around in third party articles.

Note: The spark context is language specific, and we will be looking at the python SparkContext. That being said, I think it's safe to say most of what we say about the python API will apply to the other language bindings.

I decided to have a look at the [source code](https://github.com/apache/spark/blob/e91ef1929201d4e493bb451fef0fb1b45800adae/python/pyspark/context.py#L66) to really understand what this object is. In the class defintion I see some comments but they are not particularely helpful:

> Main entry point for Spark functionality. A SparkContext represents the connection to a Spark cluster, and can be used to create RDDs and broadcast variables on that cluster.

This doesn't really help tie it in with the rest of the components So I decided to look at the SparkContext object's constructor. According to the code, it does the following:

1. First the SparkContext ensures it is not running in a task on the worker node (it will raise an exception if it is). If it is not in a worker, the code assumes it is running in a driver. This phrasing was a bit confusing until I got deeper and understood how the py4j/JavaGateway worked (we will come back to this). 

2. Next the SparkContext initializes. This starts by considering a SparkConf object (and some other low level configurations for our driver and spark cluster) to constructs a Shell command. Based on the SparkConf the Shell command runs the spark-submit utility. If no target is specified (like when running from a jupyter notebook) the spark-submit utility is instructed to run the spark shell (in the case of python this is the pyspark shell). The Spark Shell acts as the Driver. Once the Spark Shell program is running, the SparkContext will determine the port it is listening on and leverage the magic of py4j. It  creates the JavaGateway and configures it so that the SparkContext can communicate with the driver (and other objects) running in the JVM. Specifically the SparkContext running in the python interpreter gets a reference to the JavaSparkContext object running in the JVM (through the marking of py4j). This Java object is what actually connects an API to the Spark cluster.

Note: I have seen a lot of confusing statements made about the driver and the spark context. For example I have seen statements that the Driver creates the SparkContext. This is a half truth. In the case of python, scala, or R, the SparkContext exists in both the JVM and the respective language. The driver does create the JavaSparkContext but not the accompanying SparkContext for the language binding. 

I have found that running multiple SparkContexts in a single JVM is not reccomended.

## 4.7. The Driver

The Driver is a java process that runs in its own JVM. The Driver utilizes several components including the DAGScheduler, TaskScheduler, BackendScheduler and BlockManager to interpret and translate the user defined code in the Application into actual Spark Jobs which can be executed on the cluster. For example, python function calls become Transformations and Actions which are types of Spark Tasks. Once translated, the driver comes up with an execution plan and schedules the work with the Cluster Manager. Additionally, the Application may create data or cache data within the cluster. The Driver is also respobsible for keeping track of these resources. 

The Driver also hosts the Spark Web UI which allows admins to monitor the utlization of the cluster by the Application.

The driver creates the SparkContext, connecting the user program to a given Spark Master

### 4.7.1. Deploy Mode
Spark offers two methods to configure how and where the Driver runs. This is referred to as Deploy Mode or Execution Mode. 

Note: Most of our examples will deal with the "Client Mode" method of using spark. This is the method by which we can leverage spark through a jupyter notebook. Even if we tell the notebok to run in cluster mode, we will see that this setting is ignored.


#### 4.7.1.1. Client Mode
In **client mode**, the driver is launched in the same process as the client that submits the application. An example of this is when using spark from a jupyter notebook. Here, the notebook spins up a SparkContext which launches the pyspark program as the driver in a subprocess. This is why we previously install java on our machines; the driver requires java.


#### 4.7.1.2. Cluster Mode

In **cluster mode** however, the driver runs somewhere on the cluster (on a worker node). Exactly where depends on the type of Cluster Manager. This is useful as it allows the client to "fire and forget". The client can submit the application, walk away and come back to a set of completed results. This method of execution also useful when one needs to minimize the network latency between the driver and the workers

# 5. System Architecture
Recall that Spark is a distributed system and thus Spark applciations are distributed appliations. In order to understand how to use Spark we need to understand what it is and how it works. First we will focus on the system architecture before discussing the software that runs on top of the system.


<center><img src="images/cluster-overview.png", width="400px"></center>

A Spark cluster is designed according to the traditional master-slave pattern. Typically, a master node is responsible for organizing, provisioning resources on, and distributing work to, the worker nodes who perform the actual computations and return results. Spark has renamed the master/slaves as cluster manager and workers respectively.

## 5.1. The Cluster Manager (The Master Node)
The Cluster Manager communicates with the SparkContext to understand what work needs to be accomplished. It takes the instructions and coordinates the execution with the worker nodes. In some cases, the Cluster Manager also provisions the instances of the worker node based on configurations passed to it from the SparkConf object.

Spark supports the following cluster managers:

- [Standalone](https://spark.apache.org/docs/latest/spark-standalone.html) – a simple cluster manager included with Spark that makes it easy to set up a cluster.
 - [Apache Mesos](https://spark.apache.org/docs/latest/running-on-mesos.html) – a general cluster manager that can also run Hadoop MapReduce and service applications. (Deprecated)
- [Hadoop YARN](https://spark.apache.org/docs/latest/running-on-yarn.html) – the resource manager in Hadoop 2.
- [Kubernetes](https://spark.apache.org/docs/latest/running-on-kubernetes.html) – an open-source system for automating deployment, scaling, and management of containerized applications.

https://spark.apache.org/docs/latest/cluster-overview.html

## 5.2. The Executor (The Worker Node)
Performs the set of operations assigned to it by the Cluster Manager and returns the result.